In [ ]:
# Блок 2 Корреляционные зависимости

In [ ]:
# Импортируем в проект требуемые библиотеки

# Блок основных библиотек
import pandas as pd # библиотека для обработки и анализа данных
import numpy as np
import seaborn as sns # библиотека для создания статистических графиков
import random #  генераторатор случайных чисел и данных
import os # библиотека функций для работы с операционной системой.

# Блок модулей и библиотек для построения графиков
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.interpolate import make_interp_spline
from scipy.interpolate import interp1d
%matplotlib inline
sns.set_style('darkgrid')
# from mpl_toolkits.mplot3d import Axes3D

# Блок библиотек и модулей для статистического и корреляционного анализа
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import scipy
from scipy import stats # для метода обнаружения Q и IQR,  а также z-корреляции
from scipy.stats import mstats
import ydata_profiling  # Модуль статистического отчета. Форма pandas_profiling!!!! отменяется c 1-го апреля 2023 г

# Блок исключеия вывода предупреждающих ошибок
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
data_main_clean = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\Diploma_var\data_main_clean.csv')


In [ ]:
data_main_clean.head().T

In [ ]:
data_main_clean.info()

# В соответствии с теорией хрупкого разрушения (теория Гриффита) прочность (σр) определяется удельной энергией (α)
# вновь образованной поверхности разрушения: (σр) = f( α * Е) где Е – модуль упругости материала
т.е. (σр) это 'Прочность при растяжении' - 'strapery_strength' (размерность в Мпа),  
а E  это 'Модуль упругости при растяжении' - 'elasticity_module_stretching'(размерность в ГПа ,т.е в 1000 раз болье чем σр)
введем еще оди признак/столбец - 'удельную энергию ' - 'energy density' (α) = σр / Е


In [ ]:
# введем новый признак ALFA
df_add_col = data_main_clean.assign(alfa = data_main_clean.strapery_strength /data_main_clean.elasticity_module_stretching)

In [ ]:
df_add_col.info()

In [ ]:
# Уменьшим размерности переменных в файле df_add_col

In [ ]:
# Функция оптимизации типов в dataframe.
# !!!! Код взят из источника https://github.com/ellavs/python-pandas-optimize-dataframe-memory-usage

def optimize_memory_usage(df, print_size=True):

# Types for optimization.
    # Типы, которые будем проверять на оптимизацию.
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # Memory usage size before optimize (Mb).
    # (RU) Размер занимаемой памяти до оптимизации (в Мб).
    before_size = df.memory_usage().sum() / 1024**2    
    for column in df.columns:
        column_type = df[column].dtypes
        if column_type in numerics:
            column_min = df[column].min()
            column_max = df[column].max()
            if str(column_type).startswith('int'):
                if column_min > np.iinfo(np.int32).min and column_max < np.iinfo(np.int32).max:
                    df[column] = df[column].astype(np.int32)
                elif column_min > np.iinfo(np.int64).min and column_max < np.iinfo(np.int64).max:
                    df[column] = df[column].astype(np.int64)  
            else:
                if column_min > np.finfo(np.float32).min and column_max < np.finfo(np.float32).max:
                    df[column] = df[column].astype(np.float32)
                else:
                    df[column] = df[column].astype(np.float64)    
    # Memory usage size after optimize (Mb).
    # (RU) Размер занимаемой памяти после оптимизации (в Мб).
    after_size = df.memory_usage().sum() / 1024**2
    if print_size: print('Memory usage size: before {:5.4f} Mb - after {:5.4f} Mb ({:.1f}%).'.format(before_size, after_size, 100 * (before_size - after_size) / before_size))
    return df

def import_data_from_csv(df):
    # Show dataframe info before optimize.
    # (RU) Показать информацию о таблице до оптимизации.
    print('-' * 80)
    print(df.info())
    print('-' * 80)
    # (RU) Оптимизация типов в dataframe.
    df = optimize_memory_usage(df) # используем функцию optimize_memory_usage , созданую выше
    # Show dataframe info after optimize.
    # (RU) Показать информацию о таблице после оптимизации.
    print('-' * 80)
    print(df.info())
    print('-' * 80)
    return df

In [ ]:
# Вызываем функцию import_data_from_csv для уменьшения размерности переменных в файле 

df_add_col = import_data_from_csv(df_add_col)

In [ ]:
df_add_col.apply(lambda x: x.nunique())

In [ ]:
df_add_col.profile_report()

In [ ]:
Продолжаем работу над исследованием нашего df - df_add_col

In [ ]:
# Начало блока исследования коэффициентов корреляции_________________________________________________________

In [ ]:
# Построим корреляционную матрицу
plt.figure(figsize=(18,8))
corr_heatmap = sns.heatmap(df_add_col.corr(), vmin=-1, vmax=1, center=0, cmap="BrBG",
                           linewidths=0.1, annot=True)
corr_heatmap.set_title('Корреляционная матрица', fontdict={'fontsize': 18}, pad=12)
plt.show()
#plt.savefig('corr_heatmap.png')

In [ ]:
# Максимальные корреляции в нащем df для полного объема строк 
print(df_add_col.corr().abs().apply(lambda x: sorted(x)[-2]))

In [ ]:
# Максимальные корреляции в нащем df для первых 100 строк 
print(df_add_col.head(100).corr().abs().apply(lambda x: sorted(x)[-2]))

In [ ]:
# Максимальные коэффициенты корреляции увеличились. 
# Исследуем влияние количества строк в выборке  на максимальные коэффициенты кореляции

In [ ]:
list(df_add_col)

In [ ]:
# Создадим DF для максимумов коэффициентов корреляции для разных значений количества  строк выборки значений от 100 до len(data_main_clean)
df_corr_value = pd.DataFrame(columns= [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650,  700, 750, 800, 850, 900, len(data_main_clean)],
                             index = list(df_add_col)
                            )

In [ ]:
df_corr_value.head(13)

# Сначала сделаем для ПЕРВЫХ N-строк, а затем сделаем для СЛУЧАЙНЫХ N -строк
# для ПЕРВЫХ N-строк

In [ ]:
# Взято df.head(column).corr().abs().apply(lambda x: sorted(x)[-2]) 
# из https://stackoverflow.com/questions/29294983/how-to-calculate-correlation-between-all-columns-and-remove-highly-correlated-on

# Функция value_corr_show заполняет df_corr_value максимальными значениями для коэффициентов корреляции по каждому параметру
# при разных значениях N -числа СЛУЧАЙНЫХ выборок из нашего DataFrame от 100 до 1000
# df - наш основной исследуемый DF
# df_corr_value - df для максимальных значений коэффициентов корреляции 
# при разных значениях N - количества СЛУЧАЙНЫХ  выбраных значений из DF

def value_corr_show(df, df_corr_value):
    for column in df_corr_value.columns:
        df_corr_value[column] = df.head(column).corr().abs().apply(lambda x: sorted(x)[-2]).round(3)
    return(df_corr_value)

In [ ]:
value_corr_show(df_add_col, df_corr_value)

In [ ]:
df_corr_value.isna().sum()  # в df  есть пропущенные значения в параметре pattern_angle

In [ ]:
df_corr_value.fillna(0.082, inplace= True) #.round(3) # Заменили занчения Nan на 0.082 - ближайшее известное значение

In [ ]:
df_corr_value.isna().sum()

In [ ]:
# Рассмотрим Графики изменения размера корреляции от количества  N строк выборки

In [ ]:
print('Графики изменения размереа корреляции от количества N строк выборки')
fig, axs = plt.subplots(1, 1, figsize=(12, 7))
for index in range(len(df_corr_value)):
    if index  == 13:
        continue
    x = df_corr_value.columns 
    y = df_corr_value.iloc[index]
    plt.plot( x, y, label= df_corr_value.index[index])
    plt.xlabel('Количество элементов в выборке N', fontsize=10, color='blue')
    plt.ylabel('Коэффициент корреляции МАХ', fontsize=10, color='blue')
    plt.legend()
    plt.grid(True)   
    # plt.show() # если включить , то будут рисоваться все 13 графиков на отдельных фреймах
      

In [ ]:
plt.figure(figsize = (20,10))
plt.title('Графики изменения размера корреляции от количества  N строк выборки')
a = 5 # количество строк
b = 3 # количество столцбцов
c = 1 # инициализация plot counter
for index in range(len(df_corr_value)):
    plt.subplot(a, b, c)
    
    x = df_corr_value.columns 
    y = df_corr_value.iloc[index]
    plt.plot( x, y, label= df_corr_value.index[index])
    plt.xlabel('Количество элементов в выборке N', fontsize=5, color='blue')
    plt.ylabel('Коэффициент корреляции МАХ', fontsize=5, color='blue')
    plt.legend()
    plt.grid(True) 
    c = c +1   

Рассмотрим более внимательно каждый график отдельно
Нарисуем плавный график используюя модуль сглаживания X_Y_Spline = make_interp_spline(x, y)
или кубической функции cubic_interpolation_model = interp1d(x, y, kind = "cubic"
plt.title('Графики изменения размера корреляции параметров от количества  N строк выборки')
index_param = df_corr_value.index.tolist()

for i in range(len(df_corr_value)):
  
    x = df_corr_value.columns 
    y = df_corr_value.iloc[i]
    '''
    # Плавная кривая сплайна
    X_Y_Spline = make_interp_spline(x, y) 
    # Возвращает равномерно распределенные числа
    # через указанный интервал - 50
    X_ = np.linspace(x.min(), x.max(), 50)
    Y_ = X_Y_Spline(X_)
    '''
    # Плавная кубическая кривая сплайна
    cubic_interpolation_model = interp1d(x, y, kind = "cubic")
    X_=np.linspace(x.min(), x.max(), 500)
    Y_=cubic_interpolation_model(X_)
    print('Параметр - ', index_param[i])
    plt.plot(X_, Y_)
    plt.xlabel('Количество элементов в выборке N', fontsize=10, color='blue')
    plt.ylabel('Коэффициент корреляции МАХ', fontsize=10, color='blue')
    plt.grid(True)   
    plt.show() 

# Максимальный коэффициент корреляции  при N= 100 каждого параметра ПАДАЕТ при увеличении количества элементов в выборке df.head(N)

In [ ]:
# Максимальные корреляции в нащем df для 100  строк 
print(data_main_clean.head(100).corr().abs().apply(lambda x: sorted(x)[-2]))

In [ ]:
# Максимум корреляции у нас наблюдается между параметром 
'elasticity_module'  и  'number_hardeners'   corr = 0.365 при выборке ПЕРВЫХ N =100

In [ ]:
plt.figure(figsize=(18,8))
corr_heatmap = sns.heatmap(data_main_clean.head(100).corr(), vmin=-1, vmax=1, center=0, cmap="BrBG",
                           linewidths=0.1, annot=True)
corr_heatmap.set_title('Корреляционная матрица для N=100 ПЕРВЫХ строк', fontdict={'fontsize': 18}, pad=12)
plt.show()
#plt.savefig('corr_heatmap.png')

In [ ]:
# Максимальные корреляции в нащем df для СЛУЧАЙНЫХ 100  строк 
print(data_main_clean.sample(n=100, random_state=42).corr().abs().apply(lambda x: sorted(x)[-2]))

In [ ]:
# Построим корреляционную матрицу для СЛУЧАЙНЫХ 100 строк

In [ ]:
plt.figure(figsize=(18,8))
corr_heatmap = sns.heatmap(data_main_clean.sample(n=100, random_state=42).corr(), vmin=-1, vmax=1, center=0, cmap="BrBG",
                           linewidths=0.1, annot=True)
corr_heatmap.set_title('Корреляционная матрица для N=100 СЛУЧАЙНЫХ строк', fontdict={'fontsize': 18}, pad=12)
plt.show()
#plt.savefig('corr_heatmap.png')

In [ ]:
df_for_lineReg = data_main_clean.sample(n=100, random_state=42)

In [ ]:
df_for_lineReg

In [ ]:
df_for_lineReg.info()

In [ ]:
df_for_lineReg.sort_index(ascending=True, inplace= True) # Сортируем по возрастанию индексы

In [ ]:
df_for_lineReg

In [ ]:
df_for_lineReg_test = data_main_clean.sample(n=100, random_state=42)

In [ ]:
# Рассмотрим другие типы графиков для исследования . 

In [ ]:
# График зависимости парметров друг от друга
# Взято из https://pyprog.pro/sns/sns_6_visualization_of_dist.html
# При построении точечных диаграмм зависимостей используем график прямой регрессии в правом верхнем углу.
# Наблюдения проведем при выборке 100  и 948 значений.

# Создаем экземпляр класса:
g = sns.PairGrid(df_add_col)

# задаем тип графиков над главной диагональю:это графики точек с линией регрессии
g.map_upper(sns.regplot)

# задаем тип графиков под главной диагональю: 
g.map_lower(sns.kdeplot, bw_adjust= 0.5, lw=3)

# задаем тип графиков на главной диагонали:
g.map_diag(sns.histplot, kde=True, bins = 30, fill = True);

In [ ]:
В задании к ВКР целевыми переменными заданы 'elasticity_module_stretching' и 'strapery_strength'
Проверим для каждой целевой переменной зависимости от параметров с выведением регрессионной прямой на графиках зависимости

In [ ]:
# Функция печати зависимости  параметров  от целевой переменной с выводом линии регрессии
# Параметры для передачи в функцию:
# df - наш dataframe
# N - количество случайных элементов для выборки графика
# f_x - размер в figsize по оси Х
# f_y - размер в figsize по оси Y

# name_у - исследуемая целевая переменная по оси У

def regplot_plt(df, N, fig_x, fig_y, name_Y):
    plt.figure(figsize = (fig_x, fig_y))
    plt.title('Графики при N = 100 СЛУЧАЙНЫХ значений для целевой переменной elasticity_module_stretching')
    a = 5 # количество строк
    b = 3 # количество столцбцов
    c = 1 # инициализация plot counter
    for column in df.columns:
        if column == name_Y:
            continue
        plt.subplot(a, b, c)
        sns.regplot(data = df.head(N),
                    x = df.head(N)[column],
                    y = name_Y,
                    ci=None,
                   ) # ci=None- без доверительного интервала!#x = , 
        plt.grid(True)
       # plt.show()
        c = c +1 
    return     

In [ ]:
# Построим графики при N = 100 СЛУЧАЙНЫХ значений для целевой переменной 'elasticity_module_stretching'
regplot_plt(df_add_col, N=100, fig_x= 30, fig_y = 30, name_Y=  'elasticity_module_stretching')

In [ ]:
Как видно из графиков при N = 100 между  целевой переменной 'elasticity_module_stretching'
а) практически не наблюдается корреляция с параметрами:
'pattern_angle', 'step_strip', 'density_strip', 'density', 'elasticity_module', 'content_epoxy_groups', 'flash_temperature',
 'surface_density' и 'resin_consumption'
б) корреляция наблюдается  с параметрами :
'ratio_filler_matrix','density', 'number_hardeners', 'flash_temperature', 'strapery_strength', 'alfa' 

# Вероятно, можно проводить регрессивный анализ по целевой переменной  'elasticity_module_stretching'
с параметрами 'ratio_filler_matrix','density', 'number_hardeners', 'flash_temperature', 'strapery_strength' 

In [ ]:
Проведем такой же анализ при N = 100  для второй целевой переменной 'strapery_strength'

In [ ]:
# Построим графики при N = 100 для целевой переменной 'elasticity_module_stretching'
regplot_plt(df_add_col, N=100, fig_x= 30, fig_y = 30, name_Y= 'strapery_strength')

In [ ]:
Как видно из графиков при N = 100 между  целевой переменной 'strapery_strength'
а) практически не наблюдается корреляция с параметрами:
'pattern_angle', 'step_strip', 'density_strip', 'number_hardeners', 
'ratio_filler_matrix', 'content_epoxy_groups','flash_temperature','resin_consumption'
б) корреляция наблюдается  с параметрами :
'density', 'elasticity_module',  'elasticity_module_stretching', 'alfa'

# Вероятно, можно проводить регрессивный анализ по целевой переменной  'strapery_strength'с параметрами: 
'density', 'elasticity_module',  'elasticity_module_stretching', 'alfa'

In [ ]:
# Построим корреляционную матрицу  для df_add_col
plt.figure(figsize=(18,8))
corr_heatmap = sns.heatmap(df_add_col.corr(), vmin=-1, vmax=1, center=0, cmap="BrBG",
                           linewidths=0.1, annot=True)
corr_heatmap.set_title('Корреляционная матрица', fontdict={'fontsize': 18}, pad=12)
plt.show()
#plt.savefig('corr_heatmap.png')

# Выбор признаков (фич)

In [ ]:
# Вероятно, можно проводить регрессивный анализ по целевой переменной  'strapery_strength'с параметрами: 
'density', 'elasticity_module',  'elasticity_module_stretching','alfa'

In [ ]:
# Вероятно, можно проводить регрессивный анализ по целевой переменной  'elasticity_module_stretching'
с параметрами 'ratio_filler_matrix','density', 'number_hardeners', 'strapery_strength' , 'alfa'

In [ ]:
df_add_col.describe().T

# Выбор признаков (фич)

Таким образом, предлагается взять df_add_col,  т.е. df в котором введен новый целевой пармаетр "alfa", 
как функция от двух целевых связанных параметров 'elasticity_module_stretching', 'strapery_strength'.
Удалим из рассмотрения параметр "pattern_angle', как парметр имеющий всего 2 значения и неоказывающий влияния на конечные данные.
Также, для более быстрого и более квалифицированного расчета, мы в качестве целевого параметра принимаем "alfa".
Из проведенного анализа  полного нашего df data_main и сравнения различных вариантов обработки нелинейными функциями, 
уберем парметры с минимальной или полностью отсутствующей корреляцией к нашим целевым парметрам, а именно : 'pattern_angle',
 'step_strip', 'density_strip', 'ratio_filler_matrix', 'elasticity_module','number_hardeners', 'content_epoxy_groups',
 'flash_temperature','resin_consumption'

И оставим только парметры :'density', 'number_hardeners','surface_density'

In [ ]:
df_add_col.to_csv('data_research_ML.csv', index=False, float_format="%.3f")

In [ ]:
В следующей части нашей работы мы попробуем провести анализ реграссии применительно к нашему набору данных data_research.

#  Нормализуем наш dataset по методу MinMaxskaler, чтобы привести наши данные к близким размерностям

In [ ]:
# список наших столбцов
list_data = list(data_main_clean)

In [ ]:
minmax_scaler = MinMaxScaler() # вызываем метод MinMaxskaler

In [ ]:
#  обучаем MinMaxScaler() -передаем только числовые значения столбцов  ,указанных списком list_data или можно перечислить какие нужны
data_main_norm = minmax_scaler.fit_transform(np.array(data_main_clean[list_data])) 
                                                           

In [ ]:
data_main_norm[:1] # Проверим первую строку

In [ ]:
# создадим новый df с нормализованными данными с названием наших столбцов 
data_main_norm_df = pd.DataFrame(data = data_main_norm, columns = list_data)
data_main_norm_df.head().T

In [ ]:
data_main_norm_df.describe().T

# Запишем нормализованный очищеный от выбросов df
data_main_norm_df.to_csv('data_main_norm.csv', index=False, float_format="%.3f")